In [ ]:
import harpy

Pixel clustering

In [ ]:
from harpy.table.pixel_clustering._cluster_intensity import _export_to_ark_format as _export_to_ark_format_pixels

# output path
path = "/Users/arnedf/VIB/DATA/ark_analysis"

sdata = harpy.datasets.cluster_blobs(
    shape=(512, 512), n_cell_types=10, n_cells=100, noise_level_channels=1.2, noise_level_nuclei=1.2, seed=10
)

img_layer = "blobs_image"
labels_layer = "blobs_labels"
channels = ["lineage_0", "lineage_1", "lineage_5", "lineage_9"]

sdata = harpy.im.pixel_clustering_preprocess(
    sdata,
    img_layer=[img_layer],
    output_layer=[f"{img_layer}_preprocessed"],
    channels=channels,
    chunks=512,
    overwrite=True,
    sigma=0.0,
)

sdata, fsom, mapping = harpy.im.flowsom(
    sdata,
    img_layer=[f"{img_layer}_preprocessed"],
    output_layer_clusters=[f"{img_layer}_flowsom_clusters"],
    output_layer_metaclusters=[f"{img_layer}_flowsom_metaclusters"],
    n_clusters=20,
    random_state=111,
    fraction=0.5,
    chunks=512,
    overwrite=True,
)

sdata = harpy.tb.cluster_intensity(
    sdata,
    mapping=mapping,
    img_layer=f"{img_layer}_preprocessed",
    labels_layer=f"{img_layer}_flowsom_clusters",
    output_layer="counts_clusters",
    overwrite=True,
)
# TODO: to visualize in napari spatialdata, this counts_clusters table layer needs to be removed from the sdata, because napari-spatialdata does not support viewing sdata that contain tables not linked to elements yet.

df = _export_to_ark_format_pixels(adata=sdata["counts_clusters"], output=path)

Cell clustering

In [ ]:
from harpy.table.cell_clustering._utils import _export_to_ark_format as _export_to_ark_format_cells
from harpy.utils._keys import ClusteringKey

sdata, fsom = harpy.tb.flowsom(
    sdata,
    labels_layer_cells=[labels_layer],
    labels_layer_clusters=f"{img_layer}_flowsom_metaclusters",
    chunks=(200, 200),
    output_layer="table_cell_clustering",
    n_clusters=20,
    overwrite=True,
)

sdata = harpy.tb.weighted_channel_expression(
    sdata,
    table_layer_cell_clustering="table_cell_clustering",
    table_layer_pixel_cluster_intensity="counts_clusters",
    output_layer="table_cell_clustering",
    clustering_key=ClusteringKey._METACLUSTERING_KEY,
    overwrite=True,
)

(
    df_cell_som_cluster_count_avg,
    df_cell_som_cluster_channel_avg,
    df_cell_meta_cluster_channel_avg,
) = _export_to_ark_format_cells(sdata, table_layer="table_cell_clustering", output=path)

Next in an environment where ark is installed run the following cells in a jupyter notebook to visualize heatmaps of pixel and cell clustering:

### Pixel meta clusters

In [ ]:
%matplotlib widget

In [ ]:
import os

import matplotlib.pyplot as plt
from ark.utils.metacluster_remap_gui import MetaClusterGui, metaclusterdata_from_files
from matplotlib import rc_file_defaults

output_file = os.path.join(path, "average_intensities_SOM_clusters.csv")

rc_file_defaults()
plt.ion()


pixel_mcd = metaclusterdata_from_files(output_file, cluster_type="pixel")
pixel_mcd.output_mapping_filename = os.path.join(path, "pixel_meta_cluster_mapping.csv")
pixel_mcg = MetaClusterGui(pixel_mcd, width=17)

### Cell clustering visualization

In [ ]:
%matplotlib widget

In [ ]:
import os

import matplotlib.pyplot as plt
from ark.utils.metacluster_remap_gui import MetaClusterGui, metaclusterdata_from_files
from matplotlib import rc_file_defaults

path = "/Users/arnedf/VIB/DATA/ark_analysis"
output_file = os.path.join(path, "cell_som_cluster_count_avg.csv")

rc_file_defaults()
plt.ion()

cell_mcd = metaclusterdata_from_files(
    output_file,
    cluster_type="cell",
    # prefix_trim=pixel_cluster_col + '_'
)
cell_mcd.output_mapping_filename = os.path.join(path, "cell_meta_cluster_mapping.csv")
cell_mcg = MetaClusterGui(cell_mcd, width=17)

In [ ]:
from ark.utils.metacluster_remap_gui import colormap_helper

raw_cmap, renamed_cmap = colormap_helper.generate_meta_cluster_colormap_dict(
    cell_mcd.output_mapping_filename, cell_mcg.im_cl.cmap, cluster_type="cell"
)

### Weighted cell SOM cluster average heatmap over channels (z-scored)

In [ ]:
import pandas as pd
from ark.phenotyping import weighted_channel_comp

path = "/Users/arnedf/VIB/DATA/ark_analysis"
output_file = os.path.join(path, "cell_clustering_channel_avg.csv")

channels = df = pd.read_csv(output_file).columns
channels = [
    channel
    for channel in channels
    if channel not in ["cell_som_cluster", "cell_meta_cluster", "cell_meta_cluster_rename"]
]

weighted_channel_comp.generate_weighted_channel_avg_heatmap(
    output_file, "cell_som_cluster", channels, raw_cmap, renamed_cmap
)

### Weighted cell meta cluster average heatmap over channels (z-scored)

In [ ]:
from ark.phenotyping import weighted_channel_comp

path = "/Users/arnedf/VIB/DATA/ark_analysis"
output_file = os.path.join(path, "cell_metaclustering_channel_avg.csv")

weighted_channel_comp.generate_weighted_channel_avg_heatmap(
    output_file, "cell_meta_cluster_rename", channels, raw_cmap, renamed_cmap
)